In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS
from statsmodels.miscmodels.ordinal_model import OrderedModel
from scipy.stats import pearsonr
from scipy.stats import chi2

## Подгрузим и смерджим все масссивы

### demography

In [2]:
df = pd.read_csv('data/df_for_analysis_yandex_gpt_final.csv')

/var/folders/2w/pb1tg6913llbt6y2rwws4nb00000gn/T/ipykernel_83316/3476257242.py:1: DtypeWarning: Columns (276,278,336,337,338,339,340) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/df_for_analysis_yandex_gpt_final.csv')


In [3]:
df.rename(columns={'country': "code"}, inplace=True)

In [4]:
# Словарь соответствия кодов и названий стран
country_map = {
    "AND": "Andorra",
    "ARG": "Argentina",
    "ARM": "Armenia",
    "BGD": "Bangladash",
    "BOL": "Bolivia",
    "BRA": "Brazil",
    "CAN": "Canada",
    "CHL": "Chile",
    "CHN": "China",
    "COL": "Colombia",
    "CYP": "Cyprus",
    "CZE": "Czechia",
    "DEU": "Germany",
    "ECU": "Ecuador",
    "EGY": "Egypt",
    "ETH": "Ethiopia",
    "GBR": "Great Britain",
    "GRC": "Greece",
    "GTM": "Guatemala",
    "HKG": "Hong Kong",
    "IDN": "Indonesia",
    "IND": "India",
    "IRN": "Iran",
    "IRQ": "Iraq",
    "JOR": "Jordan",
    "JPN": "Japan",
    "KAZ": "Kazakhstan",
    "KEN": "Kenya",
    "KGZ": "Kyrgyzstan",
    "KOR": "South Korea",
    "LBN": "Lebanon",
    "LBY": "Libya",
    "MAC": "Macao",
    "MDV": "Maldives",
    "MEX": "Mexico",
    "MMR": "Myanmar",
    "MNG": "Mongolia",
    "MOR": "Morocco",
    "MYS": "Malaysia",
    "NGA": "Nigeria",
    "NIC": "Nicaragua",
    "NIR": "Northern Ireland",
    "NLD": "Netherlands",
    "NZL": "New Zealand",
    "PAK": "Pakistan",
    "PER": "Peru",
    "PHL": "Philippines",
    "PRI": "Puerto Rico",
    "ROU": "Romania",
    "RUS": "Russian Federation",
    "SGP": "Singapore",
    "SRB": "Serbia",
    "SVK": "Slovakia",
    "THA": "Thailand",
    "TJK": "Tajikistan",
    "TUN": "Tunisia",
    "TUR": "Turkey",
    "TWN": "Taiwan",
    "UKR": "Ukraine",
    "URY": "Uruguay",
    "USA": "United States",
    "UZB": "Uzbekistan",
    "VEN": "Venezuela",
    "VNM": "Vietnam",
    "ZWE": "Zimbabwe"
}

# Добавим новый столбец в DataFrame df
df['country'] = df['code'].map(country_map)

In [5]:
columns_to_keep = [
    "A_YEAR", "code", "country", 'X003R2',
    'Q260', 'Q275', 'Q288R', 'Q287', 'Q272', 'Q273', 'Q274', 'Q289',
    "Q158", "Q159", "Q160", "Q161", "Q162",
    "Q177", "Q178", "Q179", "Q180", "Q181", "Q182", "Q183", "Q184", "Q185",
    "Q186", "Q187", "Q188", "Q189", "Q190", "Q191", "Q192", "Q193", "Q194", "Q195",
    "Q158_gpt", "Q159_gpt", "Q160_gpt", "Q161_gpt", "Q162_gpt",
    "Q177_gpt", "Q178_gpt", "Q179_gpt", "Q180_gpt", "Q181_gpt", "Q182_gpt", "Q183_gpt",
    "Q184_gpt", "Q185_gpt", "Q186_gpt", "Q187_gpt", "Q188_gpt", "Q189_gpt", "Q190_gpt",
    "Q191_gpt", "Q192_gpt", "Q193_gpt", "Q194_gpt", "Q195_gpt",
    "Q158_yandex", "Q159_yandex", "Q160_yandex", "Q161_yandex", "Q162_yandex", "Q183_yandex",
    "Q184_yandex", "Q185_yandex", "Q186_yandex", "Q187_yandex", "Q188_yandex", "Q189_yandex", "Q190_yandex",
    "Q191_yandex", "Q192_yandex", "Q193_yandex", "Q194_yandex", "Q195_yandex"
]

# Оставим только нужные столбцы
df = df[columns_to_keep]
df

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q186_yandex,Q187_yandex,Q188_yandex,Q189_yandex,Q190_yandex,Q191_yandex,Q192_yandex,Q193_yandex,Q194_yandex,Q195_yandex
0,2018,AND,Andorra,3,2,3,2,3,1270,1,...,4.0,1.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0
1,2018,AND,Andorra,2,1,7,3,2,1270,2,...,8.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0
2,2018,AND,Andorra,2,1,7,2,3,810,4,...,6.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0
3,2018,AND,Andorra,3,2,2,2,4,1270,2,...,7.0,1.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0
4,2018,AND,Andorra,2,1,2,2,4,1270,2,...,7.0,1.0,3.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95402,2020,ZWE,Zimbabwe,2,2,1,2,3,3830,1,...,3.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0
95403,2020,ZWE,Zimbabwe,2,1,6,2,2,3200,1,...,5.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0
95404,2020,ZWE,Zimbabwe,1,2,2,2,4,3830,1,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0
95405,2020,ZWE,Zimbabwe,2,2,2,1,4,3830,1,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0


In [6]:
countries = df

countries['Q189_gpt'] = pd.to_numeric(countries['Q189_gpt'], errors='coerce')
countries.dropna(subset=["Q189", "Q189_gpt"], inplace=True)

countries[["Q189", "Q189_gpt"]].applymap(lambda x: 1 <= x <= 10).all().all()

def mae(x, y):
    return np.mean(np.abs(x - y))

def rmse(x, y):
    return np.sqrt(np.mean((x - y) ** 2))

def corr_safe(x, y):
    # аккуратно обойдём случай const-вектора
    if x.nunique() < 2 or y.nunique() < 2:
        return np.nan
    return pearsonr(x, y)[0]

metrics = (countries
           .groupby("code")
           .apply(lambda g: pd.Series({
               "N": len(g),
               "corr": corr_safe(g["Q189"], g["Q189_gpt"]),
               "MAE": mae(g["Q189"], g["Q189_gpt"]),
               "RMSE": rmse(g["Q189"], g["Q189_gpt"])
           }))
           .reset_index())

metrics

/var/folders/2w/pb1tg6913llbt6y2rwws4nb00000gn/T/ipykernel_83316/1168393200.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  countries[["Q189", "Q189_gpt"]].applymap(lambda x: 1 <= x <= 10).all().all()
/var/folders/2w/pb1tg6913llbt6y2rwws4nb00000gn/T/ipykernel_83316/1168393200.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,code,N,corr,MAE,RMSE
0,AND,1004.0,NaN,0.212151,1.069777
1,ARG,1003.0,NaN,0.452642,1.396478
2,ARM,1223.0,NaN,0.255110,1.096340
3,BGD,1200.0,0.017482,0.915000,1.644182
4,BOL,2067.0,0.033391,0.755201,1.966458
...,...,...,...,...,...
60,USA,2596.0,NaN,0.440293,1.488916
61,UZB,1250.0,0.029000,1.504800,2.947677
62,VEN,1190.0,0.067454,1.097479,2.339647
63,VNM,1200.0,0.022208,1.801667,2.845757


In [7]:
good = metrics.query("corr >= 0.1 & N >= 30")["code"]
df_good = df[df["code"].isin(good)]
df_good

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q186_yandex,Q187_yandex,Q188_yandex,Q189_yandex,Q190_yandex,Q191_yandex,Q192_yandex,Q193_yandex,Q194_yandex,Q195_yandex
90794,2019,TUN,Tunisia,3,2,1,2,5,4360,5,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,6.0
90795,2019,TUN,Tunisia,3,2,0,2,4,4360,-5,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0
90796,2019,TUN,Tunisia,2,2,3,2,2,4360,1,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,4.0
90797,2019,TUN,Tunisia,3,2,0,2,3,4360,1,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0
90798,2019,TUN,Tunisia,3,1,1,2,2,4360,1,...,3.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91997,2019,TUN,Tunisia,3,2,0,2,2,4360,1,...,5.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,5.0
91998,2019,TUN,Tunisia,3,2,1,1,4,4360,1,...,3.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0
91999,2019,TUN,Tunisia,3,2,0,1,5,4360,1,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,5.0
92000,2019,TUN,Tunisia,2,1,7,1,5,4360,1,...,4.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,4.0


### gdp, infant mortality, tertiary education, internet

In [8]:
df_gpd_wvs_year = pd.read_csv('data/gdp_per_capita_by_country.csv')
df_gpd_wvs_year

,code,country,year,ВВП на душу населения (того же года)
0,AND,Andorra,2018,42819.774580
1,ARG,Argentina,2017,14532.500931
2,ARM,Armenia,2021,4685.179971
3,BGD,Bangladash,2018,1965.243727
4,BOL,Bolivia,2017,3306.303012
...,...,...,...,...
60,USA,United States,2017,60322.261424
61,UZB,Uzbekistan,2022,2578.666894
62,VEN,Venezuela,2021,NaN
63,VNM,Vietnam,2020,3534.039535


In [9]:
df_gpd_wvs_year.rename(columns={'country': "Country", 'code': 'Code'}, inplace=True)
df_gpd_wvs_year.drop(columns=['year'], inplace=True)

df_1 = df.merge(df_gpd_wvs_year, how='left', left_on='code', right_on='Code')
# Удалим дублирующий столбец, если он больше не нужен
df_1.drop(columns=['Country', 'Code'], inplace=True)

df_1.rename(columns={'ВВП на душу населения (того же года)': "GDP_wvs_year"}, inplace=True)
df_1

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q187_yandex,Q188_yandex,Q189_yandex,Q190_yandex,Q191_yandex,Q192_yandex,Q193_yandex,Q194_yandex,Q195_yandex,GDP_wvs_year
0,2018,AND,Andorra,3,2,3,2,3,1270,1,...,1.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580
1,2018,AND,Andorra,2,1,7,3,2,1270,2,...,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,42819.774580
2,2018,AND,Andorra,2,1,7,2,3,810,4,...,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,42819.774580
3,2018,AND,Andorra,3,2,2,2,4,1270,2,...,1.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580
4,2018,AND,Andorra,2,1,2,2,4,1270,2,...,1.0,3.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95394,2020,ZWE,Zimbabwe,2,2,1,2,3,3830,1,...,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1730.413489
95395,2020,ZWE,Zimbabwe,2,1,6,2,2,3200,1,...,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0,1730.413489
95396,2020,ZWE,Zimbabwe,1,2,2,2,4,3830,1,...,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0,1730.413489
95397,2020,ZWE,Zimbabwe,2,2,2,1,4,3830,1,...,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,1730.413489


In [10]:
df_infant = pd.read_excel('data/infant_mortality.xlsx')
df_infant.head(1)

,code,country,year,"Mortality rate, infant (per 1,000 live births, the year before)"
0,ARG,Argentina,2017,10.1


In [11]:
df_infant.drop(columns=['year', 'country'], inplace=True)
df_infant.rename(columns={'code': 'Code', 'Mortality rate, infant (per 1,000 live births, the year before)' : 'mortality_per_1000_births'}, inplace=True)

df_2 = df_1.merge(df_infant, how='left', left_on='code', right_on='Code')
df_2.drop(columns=['Code'], inplace=True)
df_2

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q188_yandex,Q189_yandex,Q190_yandex,Q191_yandex,Q192_yandex,Q193_yandex,Q194_yandex,Q195_yandex,GDP_wvs_year,mortality_per_1000_births
0,2018,AND,Andorra,3,2,3,2,3,1270,1,...,2.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580,3.1
1,2018,AND,Andorra,2,1,7,3,2,1270,2,...,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,42819.774580,3.1
2,2018,AND,Andorra,2,1,7,2,3,810,4,...,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,42819.774580,3.1
3,2018,AND,Andorra,3,2,2,2,4,1270,2,...,2.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580,3.1
4,2018,AND,Andorra,2,1,2,2,4,1270,2,...,3.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580,3.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95394,2020,ZWE,Zimbabwe,2,2,1,2,3,3830,1,...,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1730.413489,36.9
95395,2020,ZWE,Zimbabwe,2,1,6,2,2,3200,1,...,2.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0,1730.413489,36.9
95396,2020,ZWE,Zimbabwe,1,2,2,2,4,3830,1,...,1.0,1.0,1.0,1.0,1.0,3.0,1.0,3.0,1730.413489,36.9
95397,2020,ZWE,Zimbabwe,2,2,2,1,4,3830,1,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,1730.413489,36.9


In [12]:
df_internet = pd.read_excel('data/internet_penetration.xlsx')
df_internet.head(1)

,code,country,year,"Secure Internet servers (per 1 million people, the year before)"
0,ARG,Argentina,2017,729.84


In [13]:
df_internet.drop(columns=['year', 'country'], inplace=True)
df_internet.rename(columns={'code': 'Code', 'Secure Internet servers (per 1 million people, the year before)' : 'servers_per_million_people'}, inplace=True)

df_3 = df_2.merge(df_internet, how='left', left_on='code', right_on='Code')
df_3.drop(columns=['Code'], inplace=True)
df_3

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q189_yandex,Q190_yandex,Q191_yandex,Q192_yandex,Q193_yandex,Q194_yandex,Q195_yandex,GDP_wvs_year,mortality_per_1000_births,servers_per_million_people
0,2018,AND,Andorra,3,2,3,2,3,1270,1,...,1.0,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580,3.1,3570.00
1,2018,AND,Andorra,2,1,7,3,2,1270,2,...,1.0,1.0,1.0,1.0,3.0,1.0,2.0,42819.774580,3.1,3570.00
2,2018,AND,Andorra,2,1,7,2,3,810,4,...,1.0,1.0,1.0,1.0,3.0,1.0,2.0,42819.774580,3.1,3570.00
3,2018,AND,Andorra,3,2,2,2,4,1270,2,...,1.0,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580,3.1,3570.00
4,2018,AND,Andorra,2,1,2,2,4,1270,2,...,1.0,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580,3.1,3570.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95394,2020,ZWE,Zimbabwe,2,2,1,2,3,3830,1,...,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1730.413489,36.9,64.89
95395,2020,ZWE,Zimbabwe,2,1,6,2,2,3200,1,...,1.0,1.0,1.0,1.0,3.0,1.0,3.0,1730.413489,36.9,64.89
95396,2020,ZWE,Zimbabwe,1,2,2,2,4,3830,1,...,1.0,1.0,1.0,1.0,3.0,1.0,3.0,1730.413489,36.9,64.89
95397,2020,ZWE,Zimbabwe,2,2,2,1,4,3830,1,...,1.0,1.0,1.0,1.0,2.0,1.0,3.0,1730.413489,36.9,64.89


In [14]:
df_educ = pd.read_excel('data/tertiary_education.xlsx')
df_educ.head(1)

,code,country,year,"Tertiary education enrollment, gross %","Gross enrolment ratio for tertiary education, both sexes (%, the year before)"
0,AND,Andorra,2018,NaN,15.71


In [15]:
df_educ.drop(columns=['year', 'country', 'Tertiary education enrollment, gross %'], inplace=True)
df_educ.rename(columns={'code': 'Code', 'Gross enrolment ratio for tertiary education, both sexes (%, the year before)' : 'gross_ration_tertiary_enrollment'}, inplace=True)

df_4 = df_3.merge(df_educ, how='left', left_on='code', right_on='Code')
df_4.drop(columns=['Code'], inplace=True)
df_4

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q190_yandex,Q191_yandex,Q192_yandex,Q193_yandex,Q194_yandex,Q195_yandex,GDP_wvs_year,mortality_per_1000_births,servers_per_million_people,gross_ration_tertiary_enrollment
0,2018,AND,Andorra,3,2,3,2,3,1270,1,...,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580,3.1,3570.00,15.71
1,2018,AND,Andorra,2,1,7,3,2,1270,2,...,1.0,1.0,1.0,3.0,1.0,2.0,42819.774580,3.1,3570.00,15.71
2,2018,AND,Andorra,2,1,7,2,3,810,4,...,1.0,1.0,1.0,3.0,1.0,2.0,42819.774580,3.1,3570.00,15.71
3,2018,AND,Andorra,3,2,2,2,4,1270,2,...,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580,3.1,3570.00,15.71
4,2018,AND,Andorra,2,1,2,2,4,1270,2,...,1.0,1.0,1.0,4.0,1.0,3.0,42819.774580,3.1,3570.00,15.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95394,2020,ZWE,Zimbabwe,2,2,1,2,3,3830,1,...,1.0,1.0,1.0,3.0,1.0,2.0,1730.413489,36.9,64.89,11.13
95395,2020,ZWE,Zimbabwe,2,1,6,2,2,3200,1,...,1.0,1.0,1.0,3.0,1.0,3.0,1730.413489,36.9,64.89,11.13
95396,2020,ZWE,Zimbabwe,1,2,2,2,4,3830,1,...,1.0,1.0,1.0,3.0,1.0,3.0,1730.413489,36.9,64.89,11.13
95397,2020,ZWE,Zimbabwe,2,2,2,1,4,3830,1,...,1.0,1.0,1.0,2.0,1.0,3.0,1730.413489,36.9,64.89,11.13


In [16]:
df_fin = df_4
df_fin.to_csv("data/data_country_regressions_sci.csv")

### HDI, muslim, post-soviet

In [42]:
df.head(1)

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q186_yandex,Q187_yandex,Q188_yandex,Q189_yandex,Q190_yandex,Q191_yandex,Q192_yandex,Q193_yandex,Q194_yandex,Q195_yandex
0,2018,AND,Andorra,3,2,3,2,3,1270,1,...,4.0,1.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0,3.0


In [43]:
data = pd.read_excel('data/HDI__post_sov__muslim.xlsx')
data.head(1)

,code,country,year,HDI,post_soviet,muslim_majority,press_freedom,democratic_development,KCI,liberties_org,uai
0,AND,Andorra,2018,0.875,0,0,96.0,89.3,NaN,NaN,NaN


In [44]:
data.drop(columns=['country', 'year'], inplace=True)
data.rename(columns={'code': 'Code'}, inplace=True)

df_fin = df.merge(data, how='left', left_on='code', right_on='Code')
df_fin.drop(columns=['Code'], inplace=True)
df_fin.head(1)

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q194_yandex,Q195_yandex,HDI,post_soviet,muslim_majority,press_freedom,democratic_development,KCI,liberties_org,uai
0,2018,AND,Andorra,3,2,3,2,3,1270,1,...,1.0,3.0,0.875,0,0,96.0,89.3,NaN,NaN,NaN


## death penalty

In [ ]:
data = pd.read_excel('data/death_penalty.xlsx')
data.head(1)

,code,country,year,GPI score,Homicide Rate,Autocracy rate,has_death_penalty
0,AND,Andorra,2018.0,NaN,2.58,NaN,0


In [ ]:
data.drop(columns=['country', 'year', 'Autocracy rate'], inplace=True)
data.rename(columns={'code': 'Code', 'Homicide Rate': 'homicide_rate', 'GPI score': 'GPI'}, inplace=True)

df_fin = df.merge(data, how='left', left_on='code', right_on='Code')
df_fin.drop(columns=['Code'], inplace=True)
df_fin.head(1)

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q189_yandex,Q190_yandex,Q191_yandex,Q192_yandex,Q193_yandex,Q194_yandex,Q195_yandex,GPI,homicide_rate,has_death_penalty
0,2018,AND,Andorra,3,2,3,2,3,1270,1,...,1.0,1.0,1.0,1.0,4.0,1.0,3.0,NaN,2.58,0


## Mixed-effects regression

In [61]:
df_fin.head(1)

,A_YEAR,code,country,X003R2,Q260,Q275,Q288R,Q287,Q272,Q273,...,Q190_yandex,Q191_yandex,Q192_yandex,Q193_yandex,Q194_yandex,Q195_yandex,GDP_wvs_year,mortality_per_1000_births,servers_per_million_people,gross_ration_tertiary_enrollment
0,2018,AND,Andorra,3,2,3,2,3,1270,1,...,1.0,1.0,1.0,4.0,1.0,3.0,42819.77458,3.1,3570.0,15.71


In [ ]:
df_fin['Q158-159'] = df_fin['Q158'] + df_fin['Q159']
df_fin['Q158-159_gpt'] = df_fin['Q158_gpt'] + df_fin['Q159_gpt']

In [ ]:
df_fin_has = df_fin[df_fin['has_death_penalty'] == 1] 
df_fin_has_not = df_fin[df_fin['has_death_penalty'] == 0] 

In [17]:
df_me = df_fin
df_me = df_me[
    df_me["X003R2"].isin([1, 2, 3]) &                   # AGE: 1..3
    df_me["Q260"].isin([1, 2]) &                     # GENDER: 1 или 2
    df_me["Q275"].isin(range(0, 9)) &                # EDUCATION: 0..8
    df_me["Q288R"].isin([1, 2, 3]) &                 # INCOME: 1..3
    df_me["Q287"].isin([1, 2, 3, 4, 5]) &            # SOCIAL CLASS: 1..5             
    df_me["Q273"].isin([1, 2, 3, 4, 5, 6]) &         # MARITAL STATUS: 1..6
    df_me["Q274"].isin(range(0, 4)) &                # HOW MANY CHILDREN: 0..3
    df_me["Q289"].isin(range(0, 10))                 # RELIGIOUS: 0..9
].copy()

df_me['Q160_gpt'] = df_me['Q160_gpt'].apply(pd.to_numeric, errors='coerce')
df_me = df_me[df_me['Q160'].isin(range(1, 11))]
df_me.dropna(subset=['Q160', 'Q160_gpt', 'GDP_wvs_year'], inplace=True)
df_me['GDP_wvs_year_c'] = df_me['GDP_wvs_year'] - df_me['GDP_wvs_year'].mean()


df1 = df_me.assign(Outcome='aaaaa', Y=df_me['Q160'])
df2 = df_me.assign(Outcome='gpt', Y=df_me['Q160_gpt'])  # вторая зависимая

df_long = pd.concat([df1, df2], ignore_index=True)
preds = [
    'C(X003R2)', 'C(Q260)', 'C(Q275)', 'C(Q288R)',
    'C(Q287)', 'C(Q273)', 'C(Q274)', 'C(Q289)',
    'GDP_wvs_year_c'
]
# базовые члены
main = ' + '.join(preds)
# interaction-термы: каждый предиктор с Outcome
inter = ' + '.join(f'{p}:Outcome' for p in preds)
formula = f'Y ~ {main} + Outcome + {inter}'

model = smf.mixedlm(
    formula=formula,
    data=df_long,
    groups='code',
    re_formula="1"   # только случайный перехват
)

result = model.fit(reml=False)

print(result.summary())

                  Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      Y           
No. Observations:      139666       Method:                  ML          
No. Groups:            60           Scale:                   4.2948      
Min. group size:       854          Log-Likelihood:          -300088.5362
Max. group size:       7678         Converged:               Yes         
Mean group size:       2327.8                                            
-------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                      4.990    0.099  50.420 0.000  4.796  5.184
C(X003R2)[T.2]                 0.068    0.022   3.032 0.002  0.024  0.112
C(X003R2)[T.3]                 0.157    0.027   5.902 0.000  0.105  0.209
C(Q260)[T.2]                   0.071    0.016   4.411 0.

In [ ]:
# 1) Список имён фикс-эффектов
fe_names = result.model.exog_names  # или result.fe_params.index.tolist()
k_fe = len(fe_names)

# 2) Создаём 1×k_fe нулевую матрицу
contrast = np.zeros((1, k_fe))

# 3) Заполняем ту же строку
contrast[0, fe_names.index('GDP_wvs_year_c')] = 1
contrast[0, fe_names.index('GDP_wvs_year_c:Outcome[T.gpt]')] = 1

# 4) Тест Вальда
wald_res = result.t_test(contrast)
print(wald_res.summary())

                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0         -4.575e-06   2.04e-06     -2.239      0.025   -8.58e-06    -5.7e-07


In [ ]:
# OLS
ols = smf.ols(formula=formula, data=df_long).fit()

# MixedLM с случайным перехватом
mixed = smf.mixedlm(formula=formula, data=df_long, groups=df_long['code'], re_formula="1").fit(reml=False)

# Лог-правдоподобия моделей
llf_ols = ols.llf
llf_mixed = mixed.llf

# Статистика отношения правдоподобий (Likelihood Ratio Test)
lr_stat = 2 * (llf_mixed - llf_ols)
df_diff = 1  # число дополнительных параметров (дисперсия случайного перехвата)
p_value = chi2.sf(lr_stat, df_diff)

print(f"LR statistic: {lr_stat:.3f}")
print(f"df difference: {df_diff}")
print(f"P-value: {p_value:.3f}")

if p_value < 0.05:
    print("Случайный перехват значим – MixedLM оправдана.")
else:
    print("Нет статистической значимости случайного перехвата – можно обойтись OLS.")


LR statistic: 3953.637
df difference: 1
P-value: 0.000
Случайный перехват значим – MixedLM оправдана.


In [ ]:
preds = [
    'C(X003R2)', 'C(Q260)', 'C(Q275)', 'C(Q288R)',
    'C(Q287)', 'C(Q273)', 'C(Q274)', 'C(Q289)',
    'GDP_wvs_year_c'
]
main = ' + '.join(preds)
inter = ' + '.join(f'{p}:Outcome' for p in preds)
formula = f'Y ~ {main} + Outcome + {inter}'

# Модель FE
fe_formula = formula + ' + C(code)'
fe_res = smf.ols(fe_formula, data=df_long).fit()

# Модель RE
re_mod = smf.mixedlm(formula, data=df_long, groups=df_long['code'], re_formula="1")
re_res = re_mod.fit(reml=False)

coef_names = [name for name in fe_res.params.index if not name.startswith('C(code)')]

b_fe = fe_res.params[coef_names]
V_fe = fe_res.cov_params().loc[coef_names, coef_names]

b_re = re_res.params[coef_names]
V_re = re_res.cov_params().loc[coef_names, coef_names]

# Статистика Хаусмана
diff = b_fe - b_re
cov_diff = V_fe - V_re
stat = float(diff.values @ np.linalg.inv(cov_diff.values) @ diff.values)
p_value = chi2.sf(stat, df=len(diff))

# 7. Вывод
print(f"Hausman chi2 = {stat:.2f}, df = {len(diff)}, p-value = {p_value:.4f}")
if p_value < 0.05:
    print("Отвергаем H0: эффекты страны коррелируют с предикторами → Random Effects НЕ подходит.")
else:
    print("Не отвергаем H0: Random Effects модель адекватна.")


Hausman chi2 = -541.92, df = 72, p-value = 1.0000
Не отвергаем H0: Random Effects модель адекватна.
